In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
import math
import torch.nn as nn
softmax = nn.Softmax(dim=1)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read Data

In [ ]:
# reading excel file
HCP_data = pd.read_csv("/content/drive/MyDrive/kdag/dataset/HCP_Data_KDAG_Hackathon.xlsx - HCP_Data_KDAG_Hackathon.csv")  #change path

In [ ]:
#new date formate for time period of HCP_data
new_Time_Period = []
for data in HCP_data['Time_Period']:
  datetime_ = datetime.datetime.strptime(str(data), '%Y%m%d').date()
  new_Time_Period.append(datetime_)
HCP_data['Time_Period'] = new_Time_Period

In [ ]:
calls = HCP_data["Sales_Rep_Calls"]
samples = HCP_data["Samples_Dropped"]
emails = HCP_data["Emails_Delivered"]
brand_rx = HCP_data["Brand_Rx"]
market_rx = HCP_data["Market_Rx"]
time_period = HCP_data["Time_Period"]
speaker_program_attended = HCP_data['Speaker_Programs_Attended']
vouchers_dropped = HCP_data['Vouchers_Dropped']

In [ ]:
HCP_data.head()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,2019-01-04,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,2019-01-11,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,2019-01-18,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,2019-01-25,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,2019-02-01,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist


In [ ]:
patient_data = pd.read_csv('/content/drive/MyDrive/kdag/dataset/Patient_Data_KDAG_Hackathon.xlsx - Patient_Data_KDAG_Hackathon.csv')

In [ ]:
#to change the date formate of patient data
new_Visiting_Date = []
patient_data['Date_of_Visit'] = pd.to_datetime(patient_data['Date_of_Visit'])
for data in patient_data['Date_of_Visit']:
  new_Visiting_Date.append(data.to_pydatetime())
patient_data['Date_of_Visit'] = new_Visiting_Date

In [ ]:
patient_data.head()

,Patient ID,Physician_ID,Date_of_Visit,Year of Birth,Gender,Geographical_State
0,16445,axt01951,2020-01-12,1962.0,F,ME
1,16445,axt01951,2019-09-25,1962.0,F,ME
2,16445,axt01951,2019-02-12,1962.0,F,ME
3,16445,axt01951,2019-11-07,1962.0,F,ME
4,16445,axt01951,2019-04-17,1962.0,F,ME


In [ ]:
visit_date = patient_data["Date_of_Visit"]

In [ ]:
starting_date = []
ending_date = []
start = datetime.date(2019,1,1)
for i, date in enumerate(time_period):
  if i==57:
    break
  if i==0:
    starting_date.append(start)
  else:
    starting_date.append(time_period[i-1]+datetime.timedelta(days=1))
  ending_date.append(date)

number = []
for id in patient_data['Physician_ID']:
  temp_string = id[3:]
  if temp_string[0] == '0':
    temp_string = temp_string[1:]
  number.append(int(temp_string))

patient_data['number'] = number
temp_data = patient_data.sort_values('number',ignore_index=True)

In [ ]:
%%capture
no_of_patient_visit_in_that_week = []
i = 0
j = 0
count = 0
while i<len(HCP_data):
  doctor = HCP_data['Physician_ID'][i]
  if i%5700==0:
    print(i)
  total_patient = [0]* 57
  while j<len(temp_data) and temp_data['Physician_ID'][j]==doctor:
    date_of_visit = temp_data['Date_of_Visit'][j]
    for k in range(len(starting_date)):
      if date_of_visit>=starting_date[k] and date_of_visit<=ending_date[k]:
        total_patient[k] = total_patient[k]+1
    j=j+1
  no_of_patient_visit_in_that_week.extend(total_patient)
  i+=57

In [ ]:
HCP_data['no_of_patient_visit'] = no_of_patient_visit_in_that_week
patient_visit = HCP_data["no_of_patient_visit"]

In [ ]:
#add a new column called ratio = brand_rx/market_rx
ratio = []
for index in range(len(HCP_data)):
  if HCP_data['Market_Rx'][0]!=0:
    ratio.append(HCP_data['Brand_Rx'][0]/HCP_data['Market_Rx'][0])
  else:
    ratio.append(0)

HCP_data['ratio'] = ratio

#Neural Network

##PREPROCESSING

1) one hot encoding and label categorical data

In [ ]:
print(HCP_data['Physician_Segment'].unique())
print(HCP_data['Specialty'].unique())

['3-Low' '1-High' '2-Medium']
['Dermatologist' 'General Physician' 'Nurse Practitioner']


In [ ]:
Physician_Segment_label = []
for index,data in enumerate(HCP_data['Physician_Segment']):
  if data == '3-Low':
    Physician_Segment_label.append(0)
  elif data == '2-Medium':
    Physician_Segment_label.append(1)
  else:
    Physician_Segment_label.append(2)

In [ ]:
HCP_data['Physician_Segment']= Physician_Segment_label

In [ ]:
Speciality_Segment_label = []
for index,data in enumerate(HCP_data['Specialty']):
  if data == 'Nurse Practitioner':
    Speciality_Segment_label.append(0)
  elif data == 'General Physician':
    Speciality_Segment_label.append(1)
  else:
    Speciality_Segment_label.append(2)

In [ ]:
HCP_data['Specialty']= Speciality_Segment_label

In [ ]:
temp_data = np.array(HCP_data)

2) Creating training data

We haven't change HCP_data as it would take time to load if something goes wrong so here temp_data is referred as HCP_data

In [ ]:
HCP_data_label = ['Physician_ID','Time_Period','Brand_Rx','Market_Rx','Sales_Rep_Calls','Samples_Dropped','Physician_Segment','Emails_Delivered','Speaker_Programs_Attended','Vouchers_Dropped','Specialty','no_of_patient_visit','ratio']

In [ ]:
c = 0
print(f'final HCP_data')
for data in HCP_data_label:
  print(f'{data} and its index {c}')
  c = c+1

final HCP_data
Physician_ID and its index 0
Time_Period and its index 1
Brand_Rx and its index 2
Market_Rx and its index 3
Sales_Rep_Calls and its index 4
Samples_Dropped and its index 5
Physician_Segment and its index 6
Emails_Delivered and its index 7
Speaker_Programs_Attended and its index 8
Vouchers_Dropped and its index 9
Specialty and its index 10
no_of_patient_visit and its index 11
ratio and its index 12


In [ ]:
p1 = 56
p2 = 0
y_train = []
X_train = []
X_test = []
list_of_doctors = []
for index,data in enumerate(temp_data):
  if index==p2:
    p2+=57
  else:
    y_train.append(data)

  if index == p1:
    p1+=57
    X_test.append(data)
    list_of_doctors.append(data[0])
  else:
    X_train.append(data)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.delete(X_train,[0,1,12],axis = 1)
X_test = np.delete(X_test,[0,1,12],axis = 1)

y_train = np.array(y_train)
y_train = np.delete(y_train,[0,1,2,3,6,8,9,10,11,12],axis = 1)
y_test = np.empty([560000, 5],dtype = int)

In [ ]:
for i in range(len(y_train)):
  for j in range(len(y_train[0])):
    if y_train[i][j]>0:
      y_train[i][j]=1

3) Removing all data with all 3 zeros</br>
temp_X_train all zero element are removed</br>
temp_y_train all zero element are removed</br>
temp_label have only labels

In [ ]:
temp_X_train = []
temp_y_train = []
for index,data in enumerate(y_train):
  if data[0]==0 and data[1]==0 and data[2]==0:
    continue
  else:
    temp_X_train.append(X_train[index])
    temp_y_train.append(y_train[index])

In [ ]:
np.sum(temp_y_train,0)

array([220267, 57860, 97815], dtype=object)

In [ ]:
temp_X_train, temp_X_val, temp_y_train, temp_y_val = train_test_split(temp_X_train, temp_y_train, test_size=0.2, random_state=42)

In [ ]:

temp_X_train = np.array(temp_X_train)
temp_y_train = np.array(temp_y_train)


temp_X_val = np.array(temp_X_val)
temp_y_val = np.array(temp_y_val)


In [ ]:
label_X_train = ['Brand_Rx','Market_Rx','Sales_Rep_Calls','Samples_Dropped','Physician_Segment','Emails_Delivered','Speaker_Programs_Attended','Vouchers_Dropped','Specialty','no_of_patient_visit']

label_X_val = ['Brand_Rx','Market_Rx','Sales_Rep_Calls','Samples_Dropped','Physician_Segment','Emails_Delivered','Speaker_Programs_Attended','Vouchers_Dropped','Specialty','no_of_patient_visit']


In [ ]:
label_y_train = ['Sales_Rep_Calls','Samples_Dropped','Emails_Delivered']

label_y_val = ['Sales_Rep_Calls','Samples_Dropped','Emails_Delivered']

##Neural Network Training


Tensor Board

1) Dataset and DataLoader

In [ ]:
class timeSeriesDataset(Dataset):

  def __init__(self,final_features,final_classification):
    self.x = torch.from_numpy(final_features.astype('float64'))
    self.y = torch.from_numpy(final_classification.astype('float64'))
    self.n_samples = len(self.x)

  def __len__(self):
    return self.n_samples

  def __getitem__(self,index):
    return (self.x[index],self.y[index])


In [ ]:
batch_size = 128
train_dataset = timeSeriesDataset(temp_X_train,temp_y_train)
val_dataset = timeSeriesDataset(temp_X_val,temp_y_val)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True , num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True , num_workers=2)

test_dataset = timeSeriesDataset(X_test,y_test)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size, shuffle=True , num_workers=2)

In [ ]:
type(temp_X_train)

numpy.ndarray

2) Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

Device available for running: 
cpu


In [ ]:
#hyper - parameters
input_size = len(X_train[0])
hidden_size = 24
num_classes = 3
num_epochs = 100
learning_rate = 0.001
total_samples = len(train_dataset)
num_iterations = int(total_samples/batch_size)

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()
    self.l1 = nn.Linear(input_size,hidden_size)
    self.l2 = nn.Linear(hidden_size,hidden_size)
    self.relu = nn.ReLU()
    self.l3 = nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l3(out)
    return out

In [ ]:
model = NeuralNet(input_size,hidden_size,num_classes)

In [ ]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
#finally CrossEntropy loss applies the softmax function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

3) Training

In [ ]:
def evaluate1(labels,outputs):
  sum = 0
  for index in range(len(labels)):
    output = outputs[index]
    label = labels[index]
    total_promotion_used = int(torch.sum(label))

    if total_promotion_used !=0:
      valueTemp, indexTemp = torch.kthvalue(output, total_promotion_used)
      for index2,temp in enumerate(output):
        if temp>=valueTemp:
          output[index2]=1
        else:
          output[index2]=0
    else:
      output = (output>0.5).float()

    correct = (output == label).float()
    if 0 not in correct:
      sum+=1
  return sum

In [ ]:
def evaluate2(labels,outputs):
  sum=0
  for index in range(len(labels)):
    output = outputs[index]
    label = labels[index]
    maximum = torch.max(output)
    final_index = -1
    for index in range(len(output)):
      if output[index]==maximum:
        final_index=index
    if label[final_index] ==1:
      sum+=1

  return sum

In [ ]:
min_valid_loss = np.inf
for e in range(num_epochs):
  train_loss = 0.0
  for data, labels in train_dataloader:
  # Transfer Data to GPU if available
    if torch.cuda.is_available():
      data, labels = data.cuda(), labels.cuda()

    # Clear the gradients
    optimizer.zero_grad()
    # Forward Pass
    target = model(data.float())
    # Find the Loss
    loss = criterion(target,labels.float())
    # Calculate gradients
    loss.backward()
    # Update Weights
    optimizer.step()
    # Calculate Loss
    train_loss += loss.item()

  valid_loss = 0.0
  model.eval()	 # Optional when not using Model Specific layer
  for data, labels in val_dataloader:
  # Transfer Data to GPU if available
    if torch.cuda.is_available():
      data, labels = data.cuda(), labels.cuda()

    # Forward Pass
    target = model(data.float())
    # Find the Loss
    loss = criterion(target,labels.float())
    # Calculate Loss
    valid_loss += loss.item()

  print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_dataloader)} \t\t Validation Loss: {valid_loss / len(val_dataloader)}')
  if min_valid_loss > valid_loss/len(val_dataloader):
      print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss/len(val_dataloader):.6f}) \t Saving The Model')
      min_valid_loss = valid_loss/len(val_dataloader)
      #saving model
      torch.save(model.state_dict(), 'saved_model.pth')



Epoch 1 		 Training Loss: 1.2141607857420518 		 Validation Loss: 1.2137572957634646
Validation Loss Decreased(inf--->1.213757) 	 Saving The Model
Epoch 2 		 Training Loss: 1.2141373360952672 		 Validation Loss: 1.2142757449552102
Epoch 3 		 Training Loss: 1.2142363124694981 		 Validation Loss: 1.213418861323274
Validation Loss Decreased(1.213757--->1.213419) 	 Saving The Model
Epoch 4 		 Training Loss: 1.213841268203856 		 Validation Loss: 1.2135219106350348
Epoch 5 		 Training Loss: 1.2140792874876734 		 Validation Loss: 1.2137483665828683
Epoch 6 		 Training Loss: 1.213980145595392 		 Validation Loss: 1.2132551154640854
Validation Loss Decreased(1.213419--->1.213255) 	 Saving The Model
Epoch 7 		 Training Loss: 1.2138183974340313 		 Validation Loss: 1.2137124463043392
Epoch 8 		 Training Loss: 1.213948152732514 		 Validation Loss: 1.213494963856715
Epoch 9 		 Training Loss: 1.2138548151740425 		 Validation Loss: 1.2139219665066698
Epoch 10 		 Training Loss: 1.2138972256236669 		 Vali

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Epoch 22 		 Training Loss: 1.2136409012927942 		 Validation Loss: 1.2133756696968503


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
Traceback (most recent call last):
Exception ignored in:   File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>self._shutdown_workers()

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
        if w.is_alive():self._shutdown_workers()

  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    if w.is_alive():    
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
AssertionErrorassert self._parent_pid == os.getpid(), 'can onl

Epoch 23 		 Training Loss: 1.2136945127044965 		 Validation Loss: 1.2138156138831615


...

Epoch 25 		 Training Loss: 1.213644569840029 		 Validation Loss: 1.2133079569708267


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():<function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Epoch 26 		 Training Loss: 1.2134973698533391 		 Validation Loss: 1.2138380369202035


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f88ab64d040>Traceback (most recent call last):

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
        
self._shutdown_workers()self._shutdown_workers()  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():

      File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
assert self._parent_pid == os.getpid(), 'can only test a child process'    
AssertionErrorif w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py"

Epoch 27 		 Training Loss: 1.213653164564586 		 Validation Loss: 1.2132614968601938
Epoch 28 		 Training Loss: 1.2136346744918711 		 Validation Loss: 1.2133436859421765
Epoch 29 		 Training Loss: 1.2136733674681996 		 Validation Loss: 1.2131878151896407
Epoch 30 		 Training Loss: 1.2135295994248267 		 Validation Loss: 1.213305650466499
Epoch 31 		 Training Loss: 1.2136056163551099 		 Validation Loss: 1.2134418541639693
Epoch 32 		 Training Loss: 1.2136006550361735 		 Validation Loss: 1.2133333820160435
Epoch 33 		 Training Loss: 1.2135727324237309 		 Validation Loss: 1.214246752111359
Epoch 34 		 Training Loss: 1.213594696037384 		 Validation Loss: 1.2129379782380767
Validation Loss Decreased(1.213119--->1.212938) 	 Saving The Model
Epoch 35 		 Training Loss: 1.213650424809869 		 Validation Loss: 1.212859074670202
Validation Loss Decreased(1.212938--->1.212859) 	 Saving The Model
Epoch 36 		 Training Loss: 1.2135253948196594 		 Validation Loss: 1.2136129670944371
Epoch 37 		 Training L

##Constraints and what to do

In [ ]:
i = 0
list_not_sales = []
while i<len(temp_data):
  X_temp = temp_data[i+5:i+57]
  X_temp1 = np.delete(X_temp,[0,1],axis = 1)
  X_temp1 = X_temp1.astype('int')
  data = np.sum(X_temp1,axis = 0)
  if data[2]>47 and X_temp1[0][4]==2:
    list_not_sales.append(X_temp[0][0])
  if data[2]>23 and X_temp1[0][4]==1:
    list_not_sales.append(X_temp[0][0])
  if data[2]>11 and X_temp1[0][4]==0:
    list_not_sales.append(X_temp[0][0])
  i+=57

In [ ]:
print(len(list_not_sales))

3767


##Neural Network Testing

In [ ]:
final_answer = []
PATH = "/content/saved_model.pth"
ann = NeuralNet(input_size,hidden_size,num_classes)
ann.load_state_dict(torch.load(PATH))
tt = torch.from_numpy(X_test.astype('float'))
predicitons = ann(tt.float())
predictions = softmax(predicitons)
index = 0

for prediction in predictions:
  #prediciton for every doctor
  data = [float(prediction[0].item()),float(prediction[1].item()),float(prediction[2].item())]
  if data[0]>data[1] and data[0]>data[2]:
    if (list_of_doctors[index] not in list_not_sales):
      final_answer.append(0)
    else:
      if data[1]>=data[2]:
        final_answer.append(1)
      else:
        final_answer.append(2)
  elif data[1]>=data[2]:
    final_answer.append(1)
  else:
    final_answer.append(2)
  index = index + 1

In [ ]:
len(final_answer)

10000

In [ ]:
zero = 0
one = 0
two = 0
for data in final_answer:
  if data == 0:
    zero = zero+1
  elif data == 1:
    one = one+1
  else:
    two =two+1

print(zero)
print(one)
print(two)

6189
1270
2541


In [ ]:
count = one + zero + two

In [ ]:
zero/count, one/count, two/count

(0.6085, 0.1444, 0.2471)

In [ ]:
df = pd.DataFrame()
df['Physician_ID'] = list_of_doctors
df['Channel'] = final_answer
df.to_csv('filename.csv', index=False)

#Saving Model

In [ ]:
File = '63percentNeuralNetwork'

In [ ]:
torch.save(model.state_dict(), File)